In [304]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from io import StringIO

In [305]:
service = Service(executable_path='../chromedriver.exe')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.get(f'https://www.investing.com/economic-calendar/')
time.sleep(2)

botton_cookies = driver.find_element(
    By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'
)
botton_cookies.click()


In [306]:
botton_signup = driver.find_element(By.XPATH, '//*[@id="PromoteSignUpPopUp"]/div[2]/i')
botton_signup.click()

In [ ]:
# Unfortunately doesn't seem possible to select dates from selenium

# botton_date = driver.find_element(
#     By.XPATH, '//*[@id="datePickerToggleBtn"]'
# )
# botton_date.click()

# startDateElem = driver.find_element(
#     By.XPATH, '//*[@id="startDate"]'
# )
# startDateElem.clear()
# startDateElem.send_keys('01/01/2020')

# endDateElem = driver.find_element(
#     By.XPATH, '//*[@id="endDate"]'
# )
# endDateElem.clear()
# endDateElem.send_keys('31/01/2020')

In [307]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(5)

In [308]:
table = driver.find_element(
    By.XPATH, '//*[@class="eCalNew eCalMainNew ecoCalCont"]'
)
dfs = pd.read_html(StringIO(table.get_attribute('innerHTML')))
df_raw = dfs[0]
df_raw

,Time,Cur.,Imp.,Event,Actual,Forecast,Previous,Unnamed: 7,Unnamed: 8
0,"Sunday, April 2, 2023","Sunday, April 2, 2023","Sunday, April 2, 2023","Sunday, April 2, 2023","Sunday, April 2, 2023","Sunday, April 2, 2023","Sunday, April 2, 2023","Sunday, April 2, 2023","Sunday, April 2, 2023"
1,19:00,AUD,NaN,Judo Bank Australia Manufacturing PMI,49.1,48.7,50.5,NaN,NaN
2,19:50,JPY,NaN,Tankan All Big Industry CAPEX (Q1),3.2%,4.9%,19.2%,NaN,NaN
3,19:50,JPY,NaN,Tankan All Small Industry CAPEX (Q1),1.4%,-9.0%,3.8%,NaN,NaN
4,19:50,JPY,NaN,Tankan Big Manufacturing Outlook Index (Q1),3,4,6,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2560,21:30,AUD,NaN,Plant/Machinery Capital Expenditure (QoQ) (Q1),3.7%,-0.1%,0.6%,NaN,NaN
2561,21:30,AUD,NaN,Private New Capital Expenditure (QoQ) (Q1),2.4%,1.3%,3.0%,NaN,NaN
2562,21:30,AUD,NaN,Retail Sales (MoM),0.0%,0.0%,0.4%,NaN,NaN
2563,21:45,CNY,NaN,Caixin Manufacturing PMI (May),50.9,50.3,49.5,NaN,NaN


In [309]:
df_filtered = df_raw[~df_raw['Cur.'].isna()] # Exclude holidays

In [310]:
df_filtered.loc[:, 'Date'] = df_filtered['Imp.'].ffill() # Get dates for all rows

C:\Users\lukas\AppData\Local\Temp\ipykernel_4944\892851624.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:, 'Date'] = df_filtered['Imp.'].ffill() # Get dates for all rows


In [311]:
df_filtered = df_filtered[df_filtered['Imp.'].isna()] # Remove rows with dates

In [312]:
flags = driver.find_elements(
    By.XPATH, '//*[@class="left flagCur noWrap"]/span'
)

countries = []
for flag in flags:
    countries.append(flag.get_attribute('title'))

df_filtered['Country'] = countries

In [313]:
df_filtered = df_filtered[~df_filtered['Actual'].isna()] # Get only indicators

In [314]:
df = df_filtered[['Time', 'Country', 'Cur.', 'Event',
                  'Actual', 'Forecast', 'Previous']]
df.index = pd.to_datetime(df_filtered['Date'])
df.index.name = None
df

,Time,Country,Cur.,Event,Actual,Forecast,Previous
2023-04-02,19:00,Australia,AUD,Judo Bank Australia Manufacturing PMI,49.1,48.7,50.5
2023-04-02,19:50,Japan,JPY,Tankan All Big Industry CAPEX (Q1),3.2%,4.9%,19.2%
2023-04-02,19:50,Japan,JPY,Tankan All Small Industry CAPEX (Q1),1.4%,-9.0%,3.8%
2023-04-02,19:50,Japan,JPY,Tankan Big Manufacturing Outlook Index (Q1),3,4,6
2023-04-02,19:50,Japan,JPY,Tankan Large Manufacturers Index (Q1),1,3,7
...,...,...,...,...,...,...,...
2023-05-31,21:30,Australia,AUD,Plant/Machinery Capital Expenditure (QoQ) (Q1),3.7%,-0.1%,0.6%
2023-05-31,21:30,Australia,AUD,Private New Capital Expenditure (QoQ) (Q1),2.4%,1.3%,3.0%
2023-05-31,21:30,Australia,AUD,Retail Sales (MoM),0.0%,0.0%,0.4%
2023-05-31,21:45,China,CNY,Caixin Manufacturing PMI (May),50.9,50.3,49.5


In [315]:
df[(df['Event'].str.contains('GDP'))]

,Time,Country,Cur.,Event,Actual,Forecast,Previous
2023-04-07,12:00,Russia,RUB,GDP Quarterly (YoY) (Q4),-2.7%,-7.0%,-3.5%
2023-04-13,02:00,United Kingdom,GBP,GDP (MoM) (Feb),0.0%,0.1%,0.4%
2023-04-13,02:00,United Kingdom,GBP,Monthly GDP 3M/3M Change (Feb),0.1%,0.0%,0.0%
2023-04-13,07:30,United Kingdom,GBP,NIESR Monthly GDP Tracker,0.1%,0.1%,0.1%
2023-04-13,20:00,Singapore,SGD,GDP (YoY) (Q1),0.1%,0.6%,2.1%
...,...,...,...,...,...,...,...
2023-05-31,08:30,Canada,CAD,GDP (QoQ) (Q1),0.8%,0.4%,0.0%
2023-05-31,08:30,Canada,CAD,GDP (YoY) (Q1),2.21%,2.80%,2.07%
2023-05-31,08:30,Canada,CAD,GDP (MoM) (Mar),0.0%,0.2%,0.1%
2023-05-31,08:30,Canada,CAD,GDP Annualized (QoQ) (Q1),3.1%,1.5%,-0.1%


In [319]:
df[(df['Country'] == 'Spain')]

,Time,Country,Cur.,Event,Actual,Forecast,Previous
2023-04-03,03:15,Spain,EUR,HCOB Spain Manufacturing PMI (Mar),51.3,50.1,50.7
2023-04-04,03:00,Spain,EUR,Spanish Unemployment Change,-48.8K,8.5K,2.6K
2023-04-05,03:00,Spain,EUR,Spanish Industrial Production (YoY) (Feb),-0.4%,-0.1%,-0.2%
2023-04-05,03:15,Spain,EUR,HCOB Spain Services PMI (Mar),59.4,57.5,56.7
2023-04-05,06:00,Spain,EUR,Spanish Consumer Confidence,67.4,NaN,71.6
2023-04-11,04:50,Spain,EUR,Spanish 12-Month Letras Auction,3.128%,NaN,3.295%
2023-04-11,04:50,Spain,EUR,Spanish 6-Month Letras Auction,2.994%,NaN,3.114%
2023-04-13,04:40,Spain,EUR,Spanish 10-Year Obligacion Auction,3.401%,NaN,3.360%
2023-04-13,05:30,Spain,EUR,Spanish 3-Year Bonos Auction,3.004%,NaN,3.433%
2023-04-13,06:00,Spain,EUR,Thomson Reuters IPSOS PCSI (MoM) (Apr),43.16,NaN,42.86


In [317]:
df.groupby('Country')['Country'].count()

Country
Australia          97
Brazil             92
Canada             91
China              60
Euro Zone         108
France             96
Germany           121
Hong Kong          22
India              58
Italy              79
Japan             163
New Zealand        69
Russia             46
Singapore          32
South Africa       49
South Korea        53
Spain              53
Switzerland        34
United Kingdom    153
United States     688
Name: Country, dtype: int64